In [1]:
import math
from tensorboardX import SummaryWriter
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import gym
import wrappers
#env_name = 'PongNoFrameskip-v4'
#env = wrappers.make_atari_deepmind(env_name, skip=4)
env_name =  'Acrobot-v1' #'CartPole-v1'
env = gym.make(env_name)

observation_shape = env.observation_space.shape
n_actions = env.action_space.n
shape =(None, ) + observation_shape
print(shape)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(None, 6)


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
# The typical imports
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

In [5]:
class EnvCreator(object):
    def __init__(self, name):
        self.name = name
    def __call__(self, ):
        return gym.make(self.name)

In [6]:
from a2c import A2CAgent
import tr_helpers
import networks
import ray
ray.init()

a2c_config = {
    'GAMMA' : 0.97,
    'TAU' : 0.9,
    'LEARNING_RATE' : 0.05,
    'NAME' : 'A2C_CartPole',
    'SCORE_TO_WIN' : 200,
    'EPISODES_TO_LOG' : 20, 
    'LIVES_REWARD' : 5,
    'GRAD_NORM' : 0.5,
    'ENTROPY_COEF' : 0.1,
    'NUM_ACTORS' : 1,
    'STEPS_NUM' : 1000,
    'TRUNCATE_GRADS' : True,
    'ENV_NAME' : env_name,
    'PPO' : False,
    'E_CLIP' : 0.1
}
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
actions_num = env.action_space.n
action_shape = env.action_space.shape
print(action_shape)
agent = A2CAgent(sess,'pong', observation_shape, actions_num, action_shape, a2c_config)
agent.train()

2019-04-28 14:46:54,030	INFO node.py:278 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-28_14-46-54_22429/logs.
2019-04-28 14:46:54,140	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:18429 to respond...
2019-04-28 14:46:54,276	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:31299 to respond...
2019-04-28 14:46:54,283	INFO services.py:798 -- Starting Redis shard with 10.0 GB max memory.
2019-04-28 14:46:54,331	INFO services.py:1360 -- Starting the Plasma object store with 13.477717606 GB memory using /dev/shm.



View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=dd4c39741ca87541cb1bd0c81b728ab44c5a7279d0e2e5b5

()
Frames per seconds:  784.1173346354894
saving next best rewards:  -481.0
Frames per seconds:  2523.566622985775
Frames per seconds:  2444.130549300962
saving next best rewards:  -447.0
Frames per seconds:  2078.9198727162784
saving next best rewards:  -239.5
Frames per seconds:  2310.081447325809
Frames per seconds:  2390.532490419185
Frames per seconds:  2514.6400919929447
Frames per seconds:  2435.346073942292
Frames per seconds:  2527.74593715686
Frames per seconds:  2511.1262249384536
Frames per seconds:  2444.489515217214
Frames per seconds:  2339.815427939299
Frames per seconds:  2432.251850296877
Frames per seconds:  2516.90355689386
Frames per seconds:  2486.3282604610577
Frames per seconds:  2083.9315383208923
Frames per seconds:  2522.126538557056
Frames per seconds:  2501.920147169528
Frames per seconds:  2525.5815521867517
Frames per seconds:  2554.429

KeyboardInterrupt: 

In [7]:
def evaluate(env,t_max=10000):
    rewards = []
    env._max_episode_steps = 9999
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(t_max):
        action = agent.get_action(s, True)
        ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(action)
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env_monitor)) for _ in range(1)]
env_monitor.close()

reset
reward: -500.0
